In [59]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.layers import Dropout, Flatten, Dense, Conv2D
from keras.applications import ResNet50
from keras.models import Model, Sequential
from keras.layers import Dense, GlobalAveragePooling2D, GlobalMaxPooling2D, MaxPooling2D
from keras.layers import BatchNormalization, Dropout
from keras import backend as K
from keras.applications.resnet50 import preprocess_input
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam

In [3]:
import pandas as pd
from scipy.misc import imread
import os
import matplotlib.pyplot as plt

join = os.path.join

In [4]:
PATH = "d:\\data\\cifar-10\\"
cifar_train_path = f'{PATH}train'
len(os.listdir(cifar_train_path))

50000

In [6]:
label_df = pd.read_csv(join(PATH, 'trainLabels.csv'))
label_df[label_df.id == 10].values[0][1]

'cat'

In [5]:
def load_cifar_10(path, label_file_name):
    """returns X_train, y_train, X_val, y_val"""
    image_file_names = os.listdir(join(path, 'train'))
    label_df = pd.read_csv(join(path, label_file_name))
    img_ids, img_labels = label_df.id, label_df.label
    data = []
    labels = []
    for image_file_name in image_file_names:
        image_file = image_file_name.split(".")[0]
        image_data = imread(join(path, 'train', image_file_name))
        image_label = label_df[label_df.id == int(image_file)].values[0][1]
        #data.append(preprocess_input(image_data, mode='torch'))
        image_data = image_data/255.0
        data.append(image_data)
        labels.append(image_label)
        #print(image_data, image_label)        
        #plt.imshow(image_data)
        #print(image_label)
    return np.array(data), np.array(labels)
        
def load_cifar_10_test(path):
    image_file_names = os.listdir(join(path, 'test'))
    print(image_file_names[-10:-1])
    data = []
    test_ids = []
    for image_file_name in image_file_names:
        id_name = image_file_name.split(".")[0]
        image_data = imread(join(path, 'test', image_file_name))
        image_data = image_data/255.0
        data.append(image_data)
        test_ids.append(id_name)
    return np.array(data), np.array(test_ids)
        
        

In [6]:
data, labels = load_cifar_10(PATH, 'trainLabels.csv')

C:\Users\anshu\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
test, test_ids = load_cifar_10_test(PATH)

In [8]:
test.shape, test_ids.shape

((300000, 32, 32, 3), (300000,))

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.1, random_state=42)

In [10]:
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
encoder.fit(labels)
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)

In [11]:
y_test.shape

(5000, 10)

In [12]:
del data
del labels

In [60]:
# Create the model
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)))
model.add(Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(128, kernel_size=(3, 3),  padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, kernel_size=(3, 3),  padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(128, kernel_size=(3, 3),  padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, kernel_size=(3, 3),  padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dense(10, activation='softmax'))
#model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
adam = Adam(lr=0.0001, decay=1e-6)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [64]:
from skimage import exposure

def adaptive_hist_equalization(img):
    img_adapteq = exposure.equalize_adapthist(img, clip_limit=0.03)
    return img_adapteq

train_datagen = ImageDataGenerator(shear_range=0.2, zoom_range=0.2, horizontal_flip=True, rotation_range=0.2)
train_datagen.fit(X_train)

In [65]:
model.fit_generator(train_datagen.flow(X_train, y_train), X_train.shape[0] // 64, epochs=100, workers=4,
                   validation_data=(X_test, y_test), callbacks=[EarlyStopping(min_delta=0.001, patience=10)])


Epoch 1/100
703/703 [==============================] - 42s 60ms/step - loss: 1.4283 - acc: 0.4891 - val_loss: 1.2740 - val_acc: 0.5502
Epoch 2/100
703/703 [==============================] - 42s 59ms/step - loss: 1.2822 - acc: 0.5399 - val_loss: 1.2358 - val_acc: 0.5608
Epoch 3/100
703/703 [==============================] - 42s 59ms/step - loss: 1.1511 - acc: 0.5870 - val_loss: 1.2159 - val_acc: 0.5774
Epoch 4/100
703/703 [==============================] - 42s 60ms/step - loss: 1.0897 - acc: 0.6098 - val_loss: 1.1246 - val_acc: 0.6016
Epoch 5/100
703/703 [==============================] - 42s 60ms/step - loss: 1.0108 - acc: 0.6423 - val_loss: 0.9103 - val_acc: 0.6768
Epoch 6/100
703/703 [==============================] - 42s 60ms/step - loss: 0.9666 - acc: 0.6583 - val_loss: 0.9470 - val_acc: 0.6774
Epoch 7/100
703/703 [==============================] - 42s 60ms/step - loss: 0.9060 - acc: 0.6801 - val_loss: 0.9129 - val_acc: 0.6846
Epoch 8/100
703/703 [==============================] - 

In [38]:
model.save('val_acc_8544.h5')

In [66]:
preds = model.predict(test)

In [67]:
preds[0]

array([7.4591017e-03, 4.4542027e-04, 1.9460749e-02, 2.2250682e-01,
       6.8381917e-01, 2.3572706e-02, 2.5662823e-04, 1.9937361e-02,
       3.6638486e-03, 1.8878268e-02], dtype=float32)

In [68]:
from keras.utils import to_categorical
preds_one_hot = to_categorical(np.argmax(preds, axis=1))

In [70]:
preds_one_hot[0]

array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0.], dtype=float32)

In [71]:
def submission(preds, test_ids, file_name):
    with open(file_name, "w") as fp:
        fp.write("id,label\n")
        for id, pred in zip(test_ids, preds):
            fp.write("{},{}\n".format(id, encoder.inverse_transform(np.array([pred]))[0]))
submission(preds_one_hot, test_ids, 'sub_3.csv')
            